In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autotime
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
import pickle
path_data = "../data"
with open(os.path.join(path_data, "ginibre_pp.dat"), "rb") as data:
    ginibre_pp = pickle.load(data, encoding="bytes")
points = ginibre_pp[0].points

In [ ]:
from hypton.point_pattern import PointPattern
from hypton.spatial_windows import BallWindow
import numpy as np
R = 100  # radius of the disk containing "points"
center = [0, 0]  # center of the disk containing "points"
window = BallWindow(center, R)  # creating ball window
intensity=1 / np.pi #intensity of Ginibre
ginibre_pp = PointPattern(points, window, intensity)

In [ ]:
ginibre_pp.dimension

In [ ]:
from hypton.structure_factor import StructureFactor
sf_ginibre = StructureFactor(ginibre_pp)

In [ ]:
from hypton.spatial_windows import BoxWindow
L = R/np.sqrt(2) #length side of the cube of restriction
bounds = np.array([[-L/2, -L/2], [L/2, L/2]]) #bounds of the cube
window = BoxWindow(bounds)
ginibre_pp_box = ginibre_pp.restrict_to_window(window)
sf_ginibre_box = StructureFactor(ginibre_pp_box)

In [ ]:
norm_k, si = sf_ginibre_box.compute_sf_scattering_intensity(
                                    max_k=10, meshgrid_size=300)



In [ ]:
exact_sf_ginibre = lambda x : 1 - np.exp(-x**2/4) #exact sf
fig = sf_ginibre_box.plot_scattering_intensity(norm_k, si, plot_type="all",
                                            exact_sf=exact_sf_ginibre,
                                            bins=60, error_bar=True,
                                            file_name="si_ginibre.pdf")

In [ ]:
r= np.linspace(0, 30, 500)
pcf_ppp = sf_ginibre.compute_pcf(method="ppp", r=r,
                                        correction="all")
pcf_ppp

In [ ]:
exact_pcf_ginibre = lambda x : 1 - np.exp(-x**2) #exact pcf
fig = sf_ginibre.plot_pcf(pcf_ppp, exact_pcf=exact_pcf_ginibre,
                    file_name="pcf_ppp_ginibre.pdf",
                    figsize=(10,6))

In [ ]:
pcf_fv = sf_ginibre.compute_pcf(method="fv", Kest=dict(rmax=30),
                                        fv=dict(method="b", spar=0.1))
pcf_fv

In [ ]:
fig = sf_ginibre.plot_pcf(pcf_fv, exact_pcf=exact_pcf_ginibre,
                    file_name="pcf_fv_ginibre.pdf",
                    figsize=(10,6))

In [ ]:
domain, pcf_fv_func = sf_ginibre.interpolate_pcf(
        r=pcf_fv["r"], pcf_r=pcf_fv["pcf"], clean=True)

In [ ]:
rmax = domain["rmax"]
norm_k = np.linspace(2,10, 1000)
norm_k, sf_Ogata = sf_ginibre.compute_sf_hankel_quadrature(pcf_fv_func,
                                                        norm_k=norm_k, 
                                                        step_size=0.1,
                                                        nb_points=1000,
                                                        rmax=rmax)
norm_k_min = sf_ginibre.norm_k_min
print("norm_k_min=", norm_k_min)

In [ ]:
fig = sf_ginibre.plot_sf_hankel_quadrature(norm_k, sf_Ogata, error_bar=True,
                             norm_k_min=norm_k_min, bins=45,
                             exact_sf=exact_sf_ginibre,
                             file_name="sf_ginibre_Ogata.pdf")

In [ ]:
rmax = domain["rmax"]
norm_k, sf_BadChou = sf_ginibre.compute_sf_hankel_quadrature(pcf_fv_func, 
                                                            method ="BaddourChouinard",
                                                            rmax=rmax, 
                                                            nb_points=1000)

fig = sf_ginibre.plot_sf_hankel_quadrature(norm_k, sf_BadChou,
                                    exact_sf=exact_sf_ginibre, file_name="sf_baddchou.pdf")

In [ ]:
from hypton.effective_hyperuniform import EffectiveHyperuniform
hyperuniformity_test = EffectiveHyperuniform(norm_k, si)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=60)
H_ginibre = hyperuniformity_test.index_H(mean_norm_k, mean_sf, i_max=9)
print("H_ginibre=", H_ginibre)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne

In [ ]:
import matplotlib.pyplot as plt
x = np.linspace(0, 5, 300)
fig=plt.figure(figsize=(10,6))
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.plot(mean_norm_k, exact_sf_ginibre(mean_norm_k), 'g', label="exact sf")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
fig.savefig("fitted_si_poisson.pdf", bbox_inches="tight")

In [ ]:
norm_k.shape


In [ ]:
f = np.polyfit(mean_norm_k, mean_sf, deg=1)

In [ ]:
f